In [1]:
import re
from collections import namedtuple

from tqdm.notebook import tqdm
import operator as op
from functools import partial
from typing import NamedTuple
from functools import reduce
mul = partial(reduce,op.mul)
assert mul(range(2,5)) == 24

def tenumerate(*args): return enumerate(tqdm(*args)) 


def flip(f):
    def inner(a,b): return f(b,a)
    return inner

In [106]:

class Amounts(NamedTuple):
    ore: int = 0
    clay: int = 0
    obsidian: int = 0
    geode: int = 0
    
    def _operate(self,other, operator):
        if isinstance(other, Amounts):
            return Amounts(*map(operator, self, other))
        else:
            return Amounts(*(operator(s,other) for s in self))
    
    def __add__(self, other): return self._operate(other,op.add)
    def __sub__(self, other): return self._operate(other,op.sub)
    def __mul__(self, other): return self._operate(other,op.mul)
    def __rsub__(self, other): return self._operate(other,flip(op.sub))
    def __truediv__(self, other): return self._operate(other,lambda a,b: 0 if a == 0 else INF if b == 0 else a/b)
    def __floordiv__(self, other): return self._operate(other,lambda a,b: 0 if a == 0 else INF if b == 0 else a//b)

INF = 1<<31

In [101]:
Amounts(1) // Amounts(0,3)

Amounts(ore='inf', clay=0, obsidian=0, geode=0)

In [3]:
BP = namedtuple("Blueprint", "ore,clay,obsidian,geode")

In [4]:
def parse(fname):
    for line in open(fname):
        if line.strip():
            _,*ns= re.findall('[0-9]+', line)
            ns = map(int,ns)
            yield BP(ore=Amounts(ore=next(ns)), 
                               clay=Amounts(ore=next(ns)),
                               obsidian=Amounts(ore=next(ns),clay=next(ns)),
                                geode=Amounts(ore=next(ns),obsidian=next(ns)))
            
bps = list(parse("test"))

In [5]:
def evaluate(moves: list, bp: BP, t=24):
    minerals = Amounts()
    robots = Amounts(ore=1)
    breakpoint()
    while t > 0:
        #build robots
        move = moves.pop(0) if len(moves) > 0 else None
        if move:
            for i in range(4):
                minerals = minerals - bp[i]*move[i]
            
            if any(m < 0 for m in minerals):
                raise ValueError(minerals)
            
            new_robots = robots + move
            
        #collect
        minerals = minerals + robots
        
        #complete robots
        if move:
            robots = new_robots
        
        print(24-t+1,minerals)
        t-=1
    return minerals

moves=[
    None,None,Amounts(clay=1),
    None,Amounts(clay=1),None,
    Amounts(clay=1),None,None,
    None,Amounts(obsidian=1),Amounts(clay=1),
    None,None,Amounts(obsidian=1),
    None,None,Amounts(geode=1),
    None,None,Amounts(geode=1),
    None,None,None
]
#evaluate(moves,bps[0])

In [6]:
def minerals_spent_building_robots(robots, bp):
    minerals = Amounts()
    for i in range(4):
        minerals = minerals + bp[i]*robots[i]   
    return minerals

def can_build_robots(robots, minerals, bp):
    minerals = minerals - minerals_spent_building_robots(robots,bp)
    return all(m >= 0 for m in minerals)


In [7]:
def valid_moves(robots, minerals, bp):    
    # just moves of one robot at a time for now
    for i in range(4):
        robot = [0,0,0,0]
        robot[i] = 1
        robot = Amounts(*robot)
        if can_build_robots(robot, minerals, bp):
            yield robot

def useful_moves(robots, minerals, bp, time_left):
    for move in valid_moves(robots, minerals, bp):
        if is_useful_move(move, robots, minerals, bp, time_left):
            yield move

def is_useful_move(move, robots, minerals, bp, time_left):
    if time_left == 1: # wont have time to reap the benefits
        return False
    elif time_left <= 3 and (move.ore or move.obsidian): # at -3 we build ore, at -2 we collect more ore, at -1 we build a geode robot, but we have no time to harvest more
        return False
    
    
    for i in range(3): # making a geode robot is always useful
        if move[i] > 0:
            # we dont need a move that make a robot if we have enough of that robot to replentish the mineral everytime
            max_of_this_mineral_we_can_spend_in_a_move = max(r[i] for r in bp)
            if robots[i] >= max_of_this_mineral_we_can_spend_in_a_move:
                return False
            
            # we don't need to make a robot if we can't possibly spend the mineral we have/will have until the end
            if (minerals[i] + robots[i]*time_left) > max_of_this_mineral_we_can_spend_in_a_move*time_left: # probably not quite right
                return False
            
    return True
            
list(valid_moves(Amounts(), Amounts(2),bps[0]))

[Amounts(ore=0, clay=1, obsidian=0, geode=0)]

In [18]:
from math import ceil

def lower_bound(robots, minerals, bp, time_left):
    return minerals.geode + robots.geode*time_left

def upper_bound_1(robots, minerals, bp, time_left):
    # assuming we can build one geode robot per round until time_left given infinite resources
    # minerals.geode + robots.geode*time_left + (time_left-1) + (time_left-2) + (time_left -2) + ...
    # the series sum(i for i in range(m)) can be simplified to (m-1)*m/2)
    return minerals.geode + robots.geode*time_left + (time_left)*(time_left-1)//2

def upper_bound_2(robots, minerals, bp, time_left):
    ub = upper_bound_1(robots, minerals, bp, time_left)
    
    # unlike upper_bound_1, we don't assume infinite resources to build geode robots. We have to take
    # into account how much obsidian we can harevest (now assuming infinite resources to build obsidian bots)
    max_obsidian = minerals.obsidian + robots.obsidian * time_left + (time_left)*(time_left-1)//2
    
    # given that we have this max obsidian, the number of geode robots we can harvest may be smaller
    max_geode_robots = ceil(max_obsidian / bp.geode.obsidian) 
    
    # now, instead of computing the sum of the series time_left -> 0, we compute the sum 
    # from time_left -> (time_left - max_geode_robots)
    if max_geode_robots < time_left:
        ub = ub - (time_left - max_geode_robots)*(time_left-max_geode_robots+1) // 2
        
    return ub

upper_bound = upper_bound_1

In [120]:
from collections import deque

def solve(bp: BP, t=24):

    best = 0
    # state is a tuple of (robots, minerals, time_left)
    todo = deque([(Amounts(ore=1), Amounts(), t)])
    visited = {}    
    state = None
    it = 0
    while todo:
        it += 1
        robots, minerals, time_left = state = todo.pop()
        
        lb = lower_bound(robots, minerals, bp, time_left)
            
        best = max(best, lb)
        
        vs = (robots[:3], minerals[:3])
        if visited.get(vs,-1) >= lb:
            continue
        else:
            visited[vs] = lb
            
        if (it % 100_000 == 0): print(it, len(todo), len(visited), time_left, best)

        if time_left == 0:
            continue
         
        if upper_bound(robots, minerals, bp, time_left) <= best:
            continue
        
        # add the do nothing move
        todo.append((robots, minerals + robots, time_left - 1))

        for move in useful_moves(robots, minerals, bp, time_left):
            new_state = (
                robots + move,
                minerals - minerals_spent_building_robots(move, bp) + robots,
                time_left - 1
            )
            todo.append(new_state)

    print(f"Got best result in {it} iterations")
    return best

In [121]:
solve(bps[1])

200000 8 134106 5 12
Got best result in 224991 iterations


12

In [47]:
ans1 = sum((i+1)*solve(bp) for i,bp in tenumerate(bps))
assert ans1 == 1150

  0%|          | 0/30 [00:00<?, ?it/s]

Got best result in 11265 iterations
Got best result in 93252 iterations
Got best result in 63853 iterations
Got best result in 36221 iterations
100000 8 65731 13 7
Got best result in 108188 iterations
100000 21 72448 3 4
Got best result in 126958 iterations
Got best result in 21564 iterations
Got best result in 39808 iterations
Got best result in 22254 iterations
Got best result in 10212 iterations
Got best result in 106531 iterations
Got best result in 134340 iterations
Got best result in 74091 iterations
Got best result in 64353 iterations
Got best result in 8590 iterations
100000 24 67658 5 2
Got best result in 203725 iterations
Got best result in 25083 iterations
Got best result in 42537 iterations
100000 12 69586 7 1
Got best result in 113504 iterations
Got best result in 96647 iterations
Got best result in 20803 iterations
100000 17 71663 5 7
Got best result in 115750 iterations
100000 14 72413 6 8
Got best result in 141961 iterations
Got best result in 60928 iterations
100000 16

In [21]:
ans2 = mul(solve(bp,t=32) for i,bp in tenumerate(bps[:3]))
ans2

  0%|          | 0/3 [00:00<?, ?it/s]

100000 17 75346 5 5
200000 11 136220 7 5
400000 22 261852 4 6
Got best result in 645525 iterations
100000 16 55041 11 56
300000 11 142343 15 63
400000 7 192576 19 69
Got best result in 423544 iterations
100000 12 64196 9 34
200000 10 112639 8 34
800000 6 397143 9 38
Got best result in 886641 iterations


36498

Idea: usar A*

https://www.reddit.com/r/adventofcode/comments/zpihwi/2022_day_19_solutions/j0tls7a/

It is still an A* and not an "explore every option" algorithm.Once you have found a way to the final day with a final geode count higher than any upper bound of an unexplored node, you can obviously abort the search and be done.
And since the upper bound of a final day node is its actual geode count, the fact that it was chosen for exploration proves that it is the best possible solution. (As the upper bound of any node can't be higher than its predecessors upper bound)

There is no assumption here, this is just the core idea of A*.
